# Docking Featurizer

This shows how we can dock a small molecule in a protein using the Hybrid docking protocol implemented in the OpenEye toolkit.

In [1]:
from kinoml.core.ligands import Ligand
from kinoml.core.proteins import BaseProtein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import OEHybridDockingFeaturizer, OEKLIFSKinaseHybridDockingFeaturizer

## OEHybridDockingFeaturizer

Let's perform a Hybrid docking into [4YNE](https://www.rcsb.org/structure/4YNE) retrieved from PDB with larotrectinib read from a smiles (reading from file is possible with the FileLigand class). First we need to generate the `System`, which is a `ProteinLigandComplex`.

In [2]:
ligand = Ligand.from_smiles(smiles="C1CC(N(C1)C2=NC3=C(C=NN3C=C2)NC(=O)N4CCC(C4)O)C5=C(C=CC(=C5)F)F",
                       name="larotrectinib")
base_protein = BaseProtein(name="4yne")
base_protein.pdb_id = "4yne"
protein_ligand_complex = ProteinLigandComplex(components=[base_protein, ligand])

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 2, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 1, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 7, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 23, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 35, aromatic: False, chiral: False
Atom atomic num: 6, name: , idx: 20, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 19, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 21, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 22, aromatic: False, chiral: False
bond ord

Next perform the docking. The current implementation will detect the co-crystallized ligand in 4YNE and perform a hybrid docking. 

> Note: The OE Spruce loop database is needed for this step. The location is specified by the `loop_db` argument. You can request a copy [here](https://www.eyesopen.com/database-downloads).

In [3]:
docking_featurizer = OEHybridDockingFeaturizer(loop_db="~/.OpenEye/rcsb_spruce.loop_db")
system = docking_featurizer.featurize([protein_ligand_complex])
system

[<ProteinLigandComplex with 2 components (<BaseProtein name=4yne>, <OpenForceFieldLigand name=larotrectinib>)>]

This docking featurizer is quite general, so it could be applied to any protein family and can be seen as a template for more advanced and protein family specific docking featurizers.

## OEKLIFSKinaseHybridDockingFeaturizer

Next, we will do the same but via the kinase specific `OEKLIFSKinaseHybridDockingFeaturizer`. This featurizer only needs a SMILES and a KLIFS kinase ID as input. It will automatically select pdb structures for generating the complex of interest.

In [4]:
ligand = Ligand.from_smiles(smiles='O=C(Nc1cnn2ccc(N3CCC[C@@H]3c3cc(F)ccc3F)nc12)N1CC[C@H](O)C1', name='larotrectinib')
base_protein = BaseProtein(name='NTRK1')
base_protein.klifs_kinase_id = 480
kinase_ligand_complex = ProteinLigandComplex(components=[base_protein, ligand])

In [5]:
docking_featurizer = OEKLIFSKinaseHybridDockingFeaturizer(loop_db="~/.OpenEye/rcsb_spruce.loop_db")
system = docking_featurizer.featurize([kinase_ligand_complex])
system

[<ProteinLigandComplex with 2 components (<BaseProtein name=NTRK1>, <OpenForceFieldLigand name=larotrectinib>)>]